In [1]:
from urllib.request import Request, urlopen as uReq
from bs4 import BeautifulSoup as soup
from selenium import webdriver
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import requests
import io
import json
import os
import pandas as pd
from config import conn_host, conn_database, conn_user, conn_password
from IPython.display import clear_output
from thefuzz import fuzz
from datetime import datetime as dt
from datetime import timedelta

C:\Users\caiog\anaconda3\lib\site-packages\thefuzz\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
league = "mls"
league_country = 'usa'
stage = 'Main'
single_year_season = True
start_season = 2018
end_season = 2024

In [3]:
option = Options()
# option.headless = True
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--disable-blink-features=AutomationControlled")
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
# driver.maximize_window()

C:\Users\caiog\AppData\Local\Temp\ipykernel_26420\3468875481.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


In [4]:
def transform_odds_date(date):
    return dt.strptime(date, '%d.%m.%Y')

seasons = dict()

for season in range(start_season, end_season):
    seasons[season] = []
    
    if single_year_season:
        season_str = f"-{season}" if season != 2023 else ""
    else:
        season_str = f"-{season}-{season+1}" if season != 2022 else ""
    url = f"https://www.betexplorer.com/football/{league_country}/{league}{season_str}/results/"
    driver.get(url)

    driver.maximize_window()
    
    try:
        if stage:
            btn = driver.find_element(By.XPATH, f"//*[contains(text(), '{stage}')]")
            btn.click()
    except: pass

    table = driver.find_element(By.XPATH, '//*[@id="js-leagueresults-all"]/div/div/table')
    rows = table.find_elements(By.XPATH, './/tbody/tr')

    total_games = 0
    for i, r in enumerate(rows):
        clear_output()
        print(f"{season}/{end_season-1} {i}/{len(rows)}")
        if not r.text: continue
        tds = r.find_elements(By.XPATH, './/child::td')
        if len(tds) < 6: continue
        matchup, score, home_odds, draw_odds, away_odds, date = [t.text for t in tds]
        
        try:
            if not score: continue
            home_score, away_score = score.split(':')

            if not matchup: continue
            home_team, away_team = matchup.split(' - ')
        
            if not date.split('.')[-1]:
                date += str(dt.now().year)
                
            match_info = [transform_odds_date(date), home_team, home_score, home_odds, away_team, away_score, away_odds, draw_odds]
            seasons[season].append(match_info)
            total_games += 1
        except Exception as e:
            continue

2023/2023 256/257


In [5]:
driver.close()

In [8]:
def set_fuzz_score(home_team, away_team, row):
    home_score = fuzz.ratio(row["home_team"], home_team)
    away_score = fuzz.ratio(row["away_team"], away_team)
    return home_score + away_score

for season in seasons.keys():
    columns = ['date', 'home_team', 'home_score', 'home_odds', 'away_team', 'away_score', 'away_odds', 'draw_odds']
    odds_df = pd.DataFrame(seasons[season], columns=columns)
    
    season_path = f"./leagues/{league}/{season}-{season+1}.csv"
    season_games = pd.read_csv(season_path, index_col=0)
    season_games["date"] = pd.to_datetime(season_games["date"])
    
    season_games["home_odds"] = None
    season_games["away_odds"] = None
    season_games["draw_odds"] = None
    
    for i, row in season_games.iterrows():
        clear_output()
        print(f"{season}/{end_season-1} : {i}/{len(season_games)}")
        
        try:
            plus_one_day = odds_df['date'] + timedelta(days=1)
            minus_one_day = odds_df['date'] - timedelta(days=1)
            same_date_matches = odds_df[(odds_df['date'] == row['date']) | (minus_one_day == row['date']) | (plus_one_day == row['date'])].reset_index(drop=True)
            same_date_matches['matchup_score'] = same_date_matches.apply(lambda x: set_fuzz_score(row['home_team'], row['away_team'], x), axis=1)
            same_date_matches = same_date_matches.sort_values(by='matchup_score', ascending=False).reset_index(drop=True)
            match = same_date_matches.iloc[0]

            season_games.at[i, "home_odds"] = match['home_odds']
            season_games.at[i, "away_odds"] = match['away_odds']
            season_games.at[i, "draw_odds"] = match['draw_odds']
            
        except:
            continue
        
    season_games.to_csv(season_path)
        

2023/2024 : 253/254


In [9]:
season_games

,date,week,home_team,home_xg,home_score,away_score,away_xg,away_team,away_Sh,away_SoT,...,home_SCA,home_Poss,home_Succ,home_Succ%,home_CrdY,home_CrdR,home_Recov,home_odds,away_odds,draw_odds
0,2023-02-25,Sat,Nashville,1.3,2,0,0.4,NYCFC,9,2,...,16,39,9,60.0,1,0,55,2.07,3.62,3.26
1,2023-02-25,Sat,FC Cincinnati,1.7,2,1,1.4,Dynamo FC,19,7,...,22,38,16,61.5,1,0,63,1.53,5.55,4.26
2,2023-02-25,Sat,FC Dallas,0.9,0,1,0.8,Minnesota Utd,8,4,...,20,62,13,76.5,1,0,48,1.81,4.42,3.50
3,2023-02-25,Sat,Atlanta Utd,1.9,2,1,1.2,San Jose,10,4,...,32,61,16,53.3,2,0,47,1.64,4.80,4.08
4,2023-02-25,Sat,Philadelphia,3.2,4,1,0.6,Columbus Crew,7,2,...,23,42,4,36.4,1,0,57,1.52,5.91,4.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,2023-06-17,Sat,D.C. United,1.2,1,2,1.8,Real Salt Lake,18,7,...,26,38,9,69.2,3,1,50,1.72,4.58,3.84
250,2023-06-17,Sat,New England,1.0,3,1,0.9,Orlando City,16,4,...,27,59,12,50.0,2,0,67,1.62,4.83,4.18
251,2023-06-17,Sat,Nashville,2.2,3,1,1.6,St. Louis,13,2,...,20,51,9,52.9,4,0,49,1.73,4.50,3.79
252,2023-06-17,Sat,San Jose,1.2,0,0,1.6,Portland Timbers,16,4,...,23,55,12,52.2,3,0,50,2.44,2.81,3.37
